In [1]:
import pandas as pd
import numpy as np

#### Realizando a leitura dos datasets

In [2]:
df_book_rating = pd.read_csv('BX-Book-Ratings.csv',sep=';',encoding = 'unicode_escape')
df_book_rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
colunas = ["ISBN","Book-Title","Book-Author","Year-Of-Publication","Publisher","Image-URL-S","Image-URL-M","Image-URL-L"]
df_book = pd.read_csv('BX-Books.csv',sep=';',names=colunas,encoding = 'unicode_escape',skiprows=1)
df_book.head()

In [ ]:
df_usuario = pd.read_csv('BX-Users.csv',sep=';',encoding = 'unicode_escape')
df_usuario.head()

#### Unindo os dados em um único dataframe

In [ ]:
df = pd.merge(pd.merge(df_book_rating,df_usuario),df_book)
df.head()

#### Substituindo os valores nulos para a o campo Age

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df.head()

#### Criando o campo de faixa_idade

In [ ]:
bins = [0,18,25,35,45,55,60]
df['faixa_idade']=pd.DataFrame(pd.cut(df['Age'].values.tolist(),bins))

#df['faixa_idade_18'] = (df['Age']<=18)
#df['faixa_idade_25'] = ((df['Age']> 18) & (df['Age']<=25))
#df['faixa_idade_35'] = ((df['Age']> 25) & (df['Age']<=35))
#df['faixa_idade_45'] = ((df['Age']> 35) & (df['Age']<=45))
#df['faixa_idade_55'] = ((df['Age']> 45) & (df['Age']<=55))
#df['faixa_idade_60'] = (df['Age']> 55)
df.head()

#### Pivotando os dados por ano de publicação e faixa de idade para plotar o gráfico

In [ ]:
total = df.pivot_table('Book-Title',index='Age',columns='faixa_idade',aggfunc='count')
total.plot(title='Tota de livros lidos por faixa de idade')

In [ ]:
total = df.pivot_table('Book-Title',index='faixa_idade',columns='Book-Rating',aggfunc='count')
total.head()

In [ ]:
subset = total[range(0,10)]
subset.plot(subplots=True,figsize=(12,10),grid=False,title='total de avaliações por faixa de idade')

In [ ]:
#separando os livros com no minimo 100 avaliações
new_df = df.groupby('ISBN').filter(lambda x: x['Book-Rating'].count ()>= 100)
new_df.head()

In [ ]:
import plotly.express as px

In [ ]:
new_df.rename(columns = {'Book-Rating': 'rating'}, inplace = True)
new_df_02 = pd.DataFrame(new_df.groupby('rating').rating.count())
new_df_02['avaliacao'] = new_df_02.index
new_df_02.rename(columns = {'rating': 'qtd_avaliacao'}, inplace = True)

In [ ]:
fig = px.bar(new_df_02, x="avaliacao", y="qtd_avaliacao", color="qtd_avaliacao", title="Quantidade de avaliações")
fig.show()

In [ ]:
# Criação de um novo DataFrame com classificação média e número de classificações por produto
ratings_df = pd.DataFrame(new_df.groupby('ISBN').rating.mean())
ratings_df['qtd_avaliacoes'] = new_df.groupby('ISBN').rating.count()
ratings_df.rename(columns = {'rating': 'media_avaliacao'}, inplace = True)
ratings_df.media_avaliacao.sort_values(ascending=False).head()

In [ ]:
popular_products = pd.DataFrame(new_df.groupby('ISBN')['rating'].count())
most_popular = popular_products.sort_values('rating', ascending=False)
most_popular.head(30).plot(kind = "bar")

In [ ]:
# Média Global das avaliações
C = ratings_df['media_avaliacao'].mean()

# Limite mínimo para ser elegível ao ranking
m = ratings_df.qtd_avaliacoes.min()

# Função que calcula a média ponderada de cada item
def weighted_rating (x, m = m, C = C):
    v = x['qtd_avaliacoes']
    R = x['media_avaliacao']
    # Calcula média ponderada
    return (v/(v+m) * R) + (m/(m+v) * C)

# Adiciona a 'pontuação' calculada com weighted_rating() ao dataframe
ratings_df['score'] = ratings_df.apply(weighted_rating, axis=1)

# Resultado final com os 15 produtos mais populares
ratings_df.sort_values(by='score', ascending=False).head()

In [ ]:
#!pip install scikit-surprise

#### Criando modelo para recomendação

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

In [ ]:
df_modelo=new_df[['User-ID','ISBN','rating']]
df_modelo.head()

In [ ]:
# Lendo o dataset
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(new_df[['User-ID','ISBN','rating']],reader)

In [ ]:
# Split dos dados 70% para treinamento e 30% para teste
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [ ]:
# Criação de um modelo baseado em item (user_based true / false para alternar entre filtragem colaborativa baseada em usuário ou baseada em item)
algo = KNNWithMeans(k=5, sim_options={'user_based': False})
algo.fit(trainset)

In [ ]:
# Teste do modelo
test_pred = algo.test(testset)

print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

In [ ]:
#Usando a função get_neighbors para obter as 10 recomendações para o produto no índice 1
algo.get_neighbors(1, 10)

In [ ]:
# Lista os 10 produtos recomendados
ratings_df.iloc[algo.get_neighbors(1, 10)].index

#### Sistema de fatoração de matriz (algoritmo SVD)

In [ ]:
new_df=new_df[['User-ID','ISBN','rating']]
ratings_matrix = new_df.pivot_table(values='rating', index='User-ID', columns='ISBN', fill_value=0)
ratings_matrix.head()


In [ ]:
# Transposta da matriz
X = ratings_matrix.T
X.head()

#### reduzirei a dimensionalidade usando Truncated SVD e calcularei a correlação entre os itens. Quanto mais os números se aproximam de 1, os itens / usuários são mais semelhantes. Quanto mais eles vão para -1, mais diferente

In [ ]:
#Decomposição da Matriz.
#A função SVD trunc irá reduzir a dimensão da matriz esparsa no número de componentes solicitados
SVD_model = TruncatedSVD(n_components=10)
decomposed_matrix = SVD_model.fit_transform(X)
decomposed_matrix.shape


In [ ]:
# Matriz de Correlação
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

In [ ]:
#indice do livro avaliado pelo cliente
X.index[75]

In [ ]:
i = "0312305060"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

In [ ]:
#Correlação para todos os livros com o livro avaliado por este cliente, com base em livros avaliados por outras pessoas
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.65])
#Remove o item avaliado pelo cliente
Recommend.remove(i)
#Lista dos 10 livros recomendados
Recommend[0:10]